In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00


In [5]:
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+${CUDA}.html

Looking in links: https://data.pyg.org/whl/torch-2.8.0+.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp312-cp312-linux_x86_64.whl size=802214 sha256=025881d971896add8bd94d538d7ffa90b5386e00a3d4b800f10815409c73bed5
  Stored in directory: /root/.cache/pip/wheels/2e/8f/d0/13408a84825c9a587151a74727b4a6d47ec67e0d625b385ad7
Successfully built torch-cluster


In [3]:
!pip install networkx

In [6]:
import torch
torch.manual_seed(100)

In [6]:
import torch_cluster

In [40]:
from torch_geometric.datasets import GeometricShapes
import networkx as nx
from torch_geometric.transforms import SamplePoints, KNNGraph
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

train_dataset = GeometricShapes(root='data/GeometricShapes', train=True)
test_dataset = GeometricShapes(root='data/GeometricShapes', train=False)

train_dataset.transform = T.Compose([SamplePoints(num=256), KNNGraph(k=6)])
test_dataset.transform = T.Compose([SamplePoints(num=256), KNNGraph(k=6)])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [10]:
#transformiere die mesh punkte zu einer point cloud
#import torch_geometric.transforms as T

#dataset.transform = T.SamplePoints(num=256)
#data = dataset[0]
#print(data)

Data(pos=[256, 3], y=[1])


In [11]:
#konvertire die einzelnen point clouds zu einem graph
#from torch_geometric.transforms import SamplePoints, KNNGraph

#dataset.transform = T.Compose([SamplePoints(num=256), KNNGraph(k=6)])
#data = dataset[0]
#print(data)

Data(pos=[256, 3], y=[1], edge_index=[2, 1536])


In [41]:
test_dataset.num_classes

40

In [42]:
from torch.nn import Sequential, Linear, ReLU
from torch import Tensor
from torch_geometric.nn import MessagePassing
import torch

class PointNetLayer(MessagePassing):
    def __init__(self, in_channels: int, out_channels: int):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')

        # Initialization of the MLP:
        # Here, the number of input features correspond to the hidden
        # node dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(
            Linear(in_channels + 3, out_channels),
            ReLU(),
            Linear(out_channels, out_channels),
        )

    def forward(self,
        h: Tensor,
        pos: Tensor,
        edge_index: Tensor,
    ) -> Tensor:
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self,
        h_j: Tensor,
        pos_j: Tensor,
        pos_i: Tensor,
    ) -> Tensor:
        # h_j: The features of neighbors as shape [num_edges, in_channels]
        # pos_j: The position of neighbors as shape [num_edges, 3]
        # pos_i: The central node position as shape [num_edges, 3]

        edge_feat = torch.cat([h_j, pos_j - pos_i], dim=-1)
        return self.mlp(edge_feat)

In [43]:
#pointNet model
from torch_geometric.nn import global_max_pool
from torch import nn

class PointNet(nn.Module):
  def __init__(self, num_classes):
    super().__init__()

    self.conv1 = PointNetLayer(3, 32)
    self.conv2 = PointNetLayer(32, 32)
    self.fc = nn.Linear(32, num_classes)

  def forward(self, data):
    pos, edge_index, batch = data.pos, data.edge_index, data.batch

    h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
    h = h.relu()
    h = self.conv2(h=h, pos=pos, edge_index=edge_index)
    h = h.relu()
    h = global_max_pool(h, batch)

    return self.fc(h)

In [44]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(device)

cpu


In [53]:
model = PointNet(40).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [54]:
#training loop
model.train()

for epoch in range(51):
  total_loss = 0
  for data in train_loader:
    data = data.to(device)
    out = model(data)
    loss = loss_fn(out, data.y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += float(loss) * data.num_graphs
  print(f"Epoch: {epoch} with loss: {total_loss / len(train_loader.dataset)}")

Epoch: 0 with loss: 3.7407586574554443
Epoch: 1 with loss: 3.697233200073242
Epoch: 2 with loss: 3.6856396794319153
Epoch: 3 with loss: 3.6660367250442505
Epoch: 4 with loss: 3.6148253083229065
Epoch: 5 with loss: 3.5553895831108093
Epoch: 6 with loss: 3.5079505443573
Epoch: 7 with loss: 3.4441736936569214
Epoch: 8 with loss: 3.3971981406211853
Epoch: 9 with loss: 3.3404136300086975
Epoch: 10 with loss: 3.2502071261405945
Epoch: 11 with loss: 3.155581533908844
Epoch: 12 with loss: 3.0885478258132935
Epoch: 13 with loss: 3.0144007205963135
Epoch: 14 with loss: 2.8703176975250244
Epoch: 15 with loss: 2.8510642647743225
Epoch: 16 with loss: 2.7847979068756104
Epoch: 17 with loss: 2.673713266849518
Epoch: 18 with loss: 2.886077582836151
Epoch: 19 with loss: 2.7033350467681885
Epoch: 20 with loss: 2.501256763935089
Epoch: 21 with loss: 2.2624512016773224
Epoch: 22 with loss: 2.318235456943512
Epoch: 23 with loss: 1.9913561344146729
Epoch: 24 with loss: 1.8796961307525635
Epoch: 25 with loss

In [55]:
#test the model
model.eval()

total_correct = 0

for data in test_loader:
  data = data.to(device)
  out = model(data)
  pred = out.argmax(dim=-1)
  total_correct += int((pred == data.y).sum())

print(total_correct / len(test_loader.dataset))

0.8
